# EEFxTMS_2F (Equatorial electric field)

> Abstract: Access to the equatorial electric field (level 2 product).

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

request = SwarmRequest()

## EEFxTMS_2F product information

Dayside equatorial electric field, sampled at every dayside equator crossing +- 20mins


Documentation:
- https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/level-2-product-definitions#EEFxTMS_2F

### Check what "EEF" data variables are available

In [ ]:
request.available_collections("EEF", details=False)

In [ ]:
request.available_measurements("EEF")

## Fetch all the EEF and EEJ values from Bravo during 2016

In [ ]:
request.set_collection("SW_OPER_EEFBTMS_2F")
request.set_products(measurements=["EEF", "EEJ", "Flags"])
data = request.get_between(
    dt.datetime(2016,1,1),
    dt.datetime(2017,1,1)
)

In [ ]:
# The first three and last three source (daily) files
data.sources[:3], data.sources[-3:]

In [ ]:
df = data.as_dataframe()
df.head()

In [ ]:
ax = df.plot(y="EEF", figsize=(20,10))
ax.set_ylim((-2, 2));
ax.set_ylabel("EEF [mV/m]");

Take a look at the time jumps between entries... Nominally the product should produce one measurement "every dayside equator crossing ±20 minutes"

In [ ]:
times = df.index
delta_t_minutes = [t.seconds/60 for t in np.diff(times.to_pydatetime())]
print("Range of time gaps (in minutes) between successive measurements:")
np.unique(np.sort(delta_t_minutes))

## Access the EEJ estimate via xarray instead of pandas

Since the EEJ estimate has both time and latitude dimensions, it is not suited to pandas. Here we load the data as a `xarray.Dataset` which better handles n-dimensional data.

In [ ]:
ds = data.as_xarray()
ds

Let's select a subset (one month) and visualise it:

In [ ]:
_ds = ds.sel({"Timestamp": "2016-01"})

fig, ax1 = plt.subplots(nrows=1, figsize=(15,3), sharex=True)
_ds.plot.scatter(x="Timestamp", y="EEJ_QDLat", hue="EEJ", vmax=10, s=1, ax=ax1)